In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import defaultdict
import numpy as np
import scipy.stats as stats 
from scipy.stats import wilcoxon

import sys
sys.path.append('/opt/miniconda3/lib/python3.8/site-packages')
from matplotlib_venn import venn3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 110)

# Change-o

To process 10X V(D)J data, a combination of AssignGenes.py and MakeDb.py can be used to generate a TSV file compliant with the AIRR Community Rearrangement schema that incorporates annotation information provided by the Cell Ranger pipeline. The --10x filtered_contig_annotations.csv specifies the path of the contig annotations file generated by cellranger vdj, which can be found in the outs directory.

In [ ]:
#sample 1
!AssignGenes.py igblast -s ./changeo_input/filtered_contig1.fasta -b ~/share/igblast \
    --organism human --loci ig --format blast --exec /Users/elinorwing/Documents/Python/ncbi-igblast-1.18.0/bin/igblastn 

In [ ]:
#sample 2
!AssignGenes.py igblast -s ./changeo_input/filtered_contig2.fasta -b ~/share/igblast \
   --organism human --loci ig --format blast  --exec /Users/elinorwing/Documents/Python/ncbi-igblast-1.18.0/bin/igblastn 

In [ ]:
#sample 3
!AssignGenes.py igblast -s ./changeo_input/filtered_contig3.fasta -b ~/share/igblast \
   --organism human --loci ig --format blast --exec /Users/elinorwing/Documents/Python/ncbi-igblast-1.18.0/bin/igblastn 

In [ ]:
!MakeDb.py igblast --log logfil.txt -i ./changeo_input/filtered_contig1_igblast.fmt7 -s ./changeo_input/filtered_contig1.fasta \
   -r /Users/elinorwing/share/germlines/imgt/human/vdj/imgt_human_*.fasta --10x ./changeo_input/filtered_contig1_annotations.csv --failed --extended 

In [ ]:
!MakeDb.py igblast --log logfil.txt -i ./changeo_input/filtered_contig2_igblast.fmt7 -s ./changeo_input/filtered_contig2.fasta \
   -r /Users/elinorwing/share/germlines/imgt/human/vdj/imgt_human_*.fasta --10x ./changeo_input/filtered_contig2_annotations.csv --failed --extended

In [ ]:
!MakeDb.py igblast --log logfile.txt -i ./changeo_input/filtered_contig3_igblast.fmt7 -s ./changeo_input/filtered_contig3.fasta \
   -r /Users/elinorwing/share/germlines/imgt/human/vdj/imgt_human_*.fasta --10x ./changeo_input/filtered_contig3_annotations.csv --failed --extended

# Import Change-o output

In [ ]:
source_folder = "/Users/elinorwing/Documents/Edinburgh/Year 2/scRNA_seq/analysis/bcr/NEW BCR/clones/input2"

file_list = [x for x in os.listdir(f'{source_folder}') if x.endswith('.tsv')]

all_df = pd.DataFrame()

for file in file_list:    
    file_df = pd.read_csv(f'{source_folder}/{file}', sep='\t')
    file_df['sample'] =  file.split("_")[0]
    all_df = all_df.append(file_df)


In [ ]:
all_df.head()

In [ ]:
all_df.groupby('sample')['cell_id'].agg(total = "nunique")

# Get only cells with one single IGH and IGK/IGL chain and get SHM

In [ ]:
#AAACCTGAGCTGTTCA-1_contig_1 split by the first '_', 0 is AAA...-1, 1 is contig_1
all_df["contig"] = all_df["sequence_id"].str.split("_", 1).str[1]

In [ ]:
#if the locus column contains 'IGH' add 'Heavy to the chain column, otherwise add 'Light'
all_df["chain"] = np.where(all_df['locus'].str.contains('IGH'), "Heavy" , "Light")

In [ ]:
chain_counts = all_df.groupby(['sample', 'chain'])['cell_id'].agg(number_chains = "value_counts")
chain_counts

In [ ]:
#new column with proper gene names - IGHV3-15*07 becomes IGHV3-15
all_df["v_gene"] = all_df["v_call"].str.split("*").str[0]
all_df["d_gene"] = all_df["d_call"].str.split("*").str[0]
all_df["j_gene"] = all_df["j_call"].str.split("*").str[0]

In [ ]:
#new column with variable chain family name - IGHV3-15*07 becomes IGHV3
all_df["v_family"] = all_df["v_call"].str.split("-").str[0]

In [ ]:
#identify duplicates where cell_id, sample, and chain are the same
multiple_chains = all_df[all_df.duplicated(subset = ["cell_id", "sample", "chain"], keep = False)].copy()

multiple_chains

In [ ]:
#remove the duplicates
all_df = all_df.drop_duplicates(subset = ["cell_id", "sample", "chain"], keep = False)

In [ ]:
#remove rows where there is no sequence alignment or germline alignment
all_df_shm = all_df.dropna(subset = ["sequence_alignment", "germline_alignment"]).copy()

In [ ]:
def get_mut_count_v(sequence, germline):
    mutations = 0
    for i, base in enumerate(sequence[:312]):
        if base != "." and base != "-" and base != "N" and germline[i] != "N" and germline[i] != "-" and germline[i] !=  "." :
            if base != germline[i]:
                mutations += 1
    return mutations
  
def get_mut_freq_v(sequence, germline):
    mutations = 0
    length = 0
    for i, base in enumerate(sequence[:312]):
        if base != "." and base != "-" and base != "N" and germline[i] != "N" and germline[i] != "-" and germline[i] !=  "." :
            length += 1
            if base != germline[i]:
                mutations += 1
    if length == 0:
        return np.nan
    else:
        return mutations/length

In [ ]:
#determine the mutation count and frequency in the variable regions using Catherines functions
all_df_shm['mut_count_v'] = all_df_shm.apply(lambda row : get_mut_count_v(row['sequence_alignment'],
                 row['germline_alignment']), axis = 1)

all_df_shm['mut_freq_v'] = all_df_shm.apply(lambda row : get_mut_freq_v(row['sequence_alignment'],
                 row['germline_alignment']), axis = 1)


all_df_shm

# Combine heavy and light annotations into single row per cell

In [ ]:
#all_df_igh contains all of all_df where the locus is IGH (heavy)
#all_df_ighlk contains the info where the locus is not IGH - IGL or IGK (light)
all_df_igh = all_df_shm.query("locus == 'IGH'")
all_df_iglk = all_df_shm.query("locus != 'IGH'")

In [ ]:
#add heavy_ to the beginning of all column names in heavy df and light_ to all columns in light df
all_df_igh = all_df_igh.add_prefix('heavy_')
all_df_iglk = all_df_iglk.add_prefix('light_')

In [ ]:
#recreate column names containing sample and cell id info for merging
all_df_igh["sample"] = all_df_igh["heavy_sample"]
all_df_iglk["sample"] = all_df_iglk["light_sample"]


all_df_igh["cell_id"] = all_df_igh["heavy_cell_id"]
all_df_iglk["cell_id"] = all_df_iglk["light_cell_id"]

In [ ]:
all_df_igh

In [ ]:
all_df_iglk

In [ ]:
#combine the two dfs
all_df_both = pd.merge(all_df_igh, all_df_iglk, how="outer", on=["cell_id", "sample"])
all_df_both

In [ ]:
all_df_both_paired = pd.merge(all_df_igh, all_df_iglk, how="inner", on=["cell_id", "sample"])
all_df_both_with_heavy = all_df_both.dropna(subset = ["heavy_sequence_id"])

all_df_both_paired

In [ ]:
all_df_both_with_heavy.to_csv("all_df_both_with_heavy_new_2.csv")
all_df_both_paired.to_csv("all_df_both_paired_new_2.csv")

In [ ]:
list(all_df_both_paired.columns)

In [ ]:
all_df_both_paired = pd.read_csv("all_df_both_paired_new.csv")

# Tidy columns and sort ordering

In [ ]:
all_df_both_paired = all_df_both_paired.sort_values(by = "sample")    
individ_order = list(all_df_both_paired["sample"].unique())
individ_order

# Number of heavy or light chains per cell

In [ ]:
chain_counts.groupby(['sample', 'chain'])["number_chains"].value_counts()

# Get percentage of all cells that have a single paired receptor

In [ ]:
all_grpd = pd.DataFrame(all_df.groupby('sample')['cell_id'].agg(total_cell_count = "nunique"))

single_pair_total = pd.DataFrame(all_df_both_paired.groupby('sample')['cell_id'].agg(total_single = "nunique").copy())

In [ ]:
all_grpd

In [ ]:
single_pair_total["total_cell_count"] = [3567, 4744, 5793]
single_pair_total

In [ ]:
single_pair_total["percent_single_pair_total"] = single_pair_total["total_single"]/single_pair_total["total_cell_count"] * 100
single_pair_total

# Output for Shazam

In [ ]:
for_shazam = all_df_shm.drop_duplicates(subset = ["cell_id", "sample", "chain"], keep = False)
for_shazam = for_shazam.dropna(subset = ["junction"])
for_shazam["cell_id_sample"] = for_shazam["cell_id"] + for_shazam["sample"]


for_shazam.to_csv("all_df_for_shazam.csv", index = False)

# Plot isotype usage per sample

In [ ]:
isotype_use = all_df_both_paired.copy()
isotype_use["count"] = 1
isotype_use = isotype_use.sort_values(by = "sample")    
individ_order = list(isotype_use["sample"].unique())

isotype_use = isotype_use[["sample", "count", "heavy_c_call"]]

isotype_use = isotype_use.groupby(['sample', 'heavy_c_call']).agg({'count': 'sum'})
isotype_use = isotype_use.groupby(level=0).apply(lambda x: x / float(x.sum())).reset_index()

isotype_use = isotype_use.pivot(index = "sample", values = "count", columns = "heavy_c_call").reset_index()

isotype_use.individ_type=pd.Categorical(isotype_use.sample,categories=individ_order)
isotype_use=isotype_use.sort_values('sample')

isotype_use

In [ ]:
f = plt.figure()

isotype_use.plot(
    x = 'sample',
    kind = 'bar',
    stacked = True,
    width = 0.9,
    figsize=(8,4),
    legend = True,
    colormap = "Set2",
    ax = f.gca());
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.xlabel("Sample")
plt.xticks(rotation=0)

# Somatic Hypermutation

In [ ]:
all_df_both_paired

In [ ]:
all_df_both_paired["heavy_mut_freq_v"] = all_df_both_paired["heavy_mut_freq_v"].astype(int)
all_df_both_paired["light_mut_freq_v"] = all_df_both_paired["light_mut_freq_v"].astype(int)

In [ ]:
all_df_both_paired_shm = all_df_both_paired.dropna(subset = ["heavy_c_call"]).copy()

In [ ]:
igg_shm_df = all_df_both_paired_shm[all_df_both_paired_shm["heavy_c_call"].str.contains("IGHG")]
iga_shm_df = all_df_both_paired_shm[all_df_both_paired_shm["heavy_c_call"].str.contains("IGHA")]
igm_shm_df = all_df_both_paired_shm[all_df_both_paired_shm["heavy_c_call"].str.contains("IGHM")]

In [ ]:
g = sns.relplot(data = igg_shm_df, x = "heavy_mut_freq_v", y = "light_mut_freq_v", alpha = 0.10, col = "heavy_sample", col_wrap = 2)
g.set_xlabels('Mutation Frequency IgH V Gene');
g.set_ylabels('Mutation Frequency IgK/L V Gene');
g.set_titles(col_template = '{col_name}');

In [ ]:
no_igd = all_df_both_paired.query("heavy_c_call!='IGHD'")

sns.displot(
    no_igd, x="heavy_mut_freq_v", hue="heavy_sample", col='heavy_c_call', col_wrap=4,
    height=4, facet_kws=dict(margin_titles=True), common_norm = False, kind='kde', 
    fill=True, palette=sns.color_palette('bright')[:3]
);

# Overlap unique heavy chain sequences

In [ ]:
set1 = set(all_df_both_paired_shm.query("heavy_sample == 1")['heavy_cdr3'])
set2 = set(all_df_both_paired_shm.query("heavy_sample == 2")['heavy_cdr3'])
set3 = set(all_df_both_paired_shm.query("heavy_sample == 3")['heavy_cdr3'])

In [ ]:
venn3([set1, set2, set3], ('Sample1', 'Sample2', 'Sample3'))
plt.show()

In [ ]:
hj1 = set(all_df_both_paired_shm.query("heavy_sample == 1")['heavy_junction_aa'])
hj2 = set(all_df_both_paired_shm.query("heavy_sample == 2")['heavy_junction_aa'])
hj3 = set(all_df_both_paired_shm.query("heavy_sample == 3")['heavy_junction_aa'])

In [ ]:
venn3([hj1, hj2, hj3], ('Sample1', 'Sample2', 'Sample3'))
plt.show()

# CDR3 Lengths

In [ ]:
all_df_both_paired = pd.read_csv("all_df_both_paired_new.csv")

In [ ]:
mean_cdr3_lengths = all_df_both_paired.copy()

mean_cdr3_lengths = mean_cdr3_lengths.groupby(["sample"])["heavy_junction_length"].mean().reset_index()

mean_cdr3_lengths

In [ ]:
mean_cdr3_lengths_iga = all_df_both_paired.dropna(subset =["heavy_c_call"]).copy()

mean_cdr3_lengths_iga = mean_cdr3_lengths_iga[mean_cdr3_lengths_iga["heavy_c_call"].str.contains("IGHA")].copy()

mean_cdr3_lengths_iga = mean_cdr3_lengths_iga.groupby(["sample"])["heavy_junction_length"].mean().reset_index()

mean_cdr3_lengths_iga

In [ ]:
mean_cdr3_lengths_igg = all_df_both_paired.dropna(subset =["heavy_c_call"]).copy()

mean_cdr3_lengths_igg = mean_cdr3_lengths_igg[mean_cdr3_lengths_igg["heavy_c_call"].str.contains("IGHG")].copy()

mean_cdr3_lengths_igg = mean_cdr3_lengths_igg.groupby(["sample"])["heavy_junction_length"].mean().reset_index()

mean_cdr3_lengths_igg

In [ ]:
mean_cdr3_lengths_igm = all_df_both_paired.dropna(subset =["heavy_c_call"]).copy()

mean_cdr3_lengths_igm = mean_cdr3_lengths_igm[mean_cdr3_lengths_igm["heavy_c_call"].str.contains("IGHM")].copy()

mean_cdr3_lengths_igm = mean_cdr3_lengths_igm.groupby(["sample"])["heavy_junction_length"].mean().reset_index()

mean_cdr3_lengths_igm

# Gini Coefficient

In [ ]:
def calculate_gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

In [ ]:
unique_cdrs = unique_cdrs.sort_values(by = "sample")

plot_gini_dict = {}

for i in unique_cdrs["sample"].unique():
    sample_df = unique_cdrs[unique_cdrs["sample"] == i]
    temp_array = sample_df.loc[:,'cell_prop'].values
    temp_array = temp_array[~np.isnan(temp_array)]
    gini = calculate_gini(temp_array)
    plot_gini_dict[i] = gini

plot_gini_df = pd.DataFrame.from_dict(plot_gini_dict, orient = "index", columns = ["gini_coeffic"]).reset_index()
plot_gini_df.rename(columns={'index': 'sample'}, inplace = True)


with sns.plotting_context("notebook", font_scale = 2):
    plt.figure(figsize = (16,8))

    g = sns.stripplot(x="sample", y="gini_coeffic", data=plot_gini_df,
                linewidth=1, s= 7, color = "black")
    sns.boxplot(x="sample",
            y="gini_coeffic",
            data=plot_gini_df,
            showfliers = False)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Gini Coefficient");

In [ ]:
plot_gini_df